# chemicalbalancer

A Python program that **_balances chemical equations_** by determining the correct stoichiometric coefficients for each reactant and product.

## Features

-   Accepts chemical equations in a **_familiar format_** (_e.g._ `Fe3O4 + HNO3 -> Fe(NO3)2 + Fe(NO3)3 + H2O`).
-   Decomposes compounds into **_individual elements_** and their respective **_counts_**.
-   Correctly processes compounds with **_parentheses_** (`()`) and **_brackets_** (`[]`) to account for **_nested groups_** and their multipliers.

## Matrix Algebra for Balancing Equations

Chemical equation balancing can be formulated as a **_system of linear equations_**, which can be solved using **_matrix algebra_**. The process involves the following steps:

### Import Modules

Firstly, import the required modules: regular expression, matrix, and least common multiple.

In [1]:
# Regular Expression
import re
# Matrix and least common multiple
from sympy import Matrix, lcm

Initialize:
-   **Element list:** Keeps track of all unique chemical elements (_e.g._ $Fe$, $O$, $H$, etc.) found in the chemical equation.
-   **Element matrix:** Represents the counts of each element in the reactants and products.

In [2]:
element_list = []
element_matrix = []

Identify the **_reactants_** and **_products_** in the input chemical equation.

In [3]:
# Get the chemical equation
equation = "Fe3O4+HNO3->Fe(NO3)2+Fe(NO3)3+H2O"
equation = re.sub(r"[ \[\]]", "", equation).split("->")

# Get the reactants and products from the equation
reactants = equation[0].split("+")
products = equation[1].split("+")

# Print the reactants and products lists
print(f"Reactants: {reactants}")
print(f"Products: {products}")

Reactants: ['Fe3O4', 'HNO3']
Products: ['Fe(NO3)2', 'Fe(NO3)3', 'H2O']


### Equation Representation as a Matrix

Each chemical equation is **_parsed_** and then **_represented_** as a **_coefficient matrix_** where:

-   **_Rows_** represent **_elements_** (_e.g._ $Fe$, $O$, $H$, etc.).
-   **_Columns_** represent **_compounds_** (reactants and products).
-   Each entry represents the **_number of atoms_** of a given **_element in a compound_**.
-   Example:

$$
Fe_3O_4 + HNO_3 \rightarrow Fe(NO_3)_2 + Fe(NO_3)_3 + H_2O
$$

The matrix is set up by **_counting atoms_** of each elements:

Element |Fe<sub>3</sub>O<sub>4</sub>|HNO<sub>3</sub>    |Fe(NO<sub>3</sub>)<sub>2</sub> |Fe(NO<sub>3</sub>)<sub>3</sub> |H<sub>2</sub>O
:------:|:-------------------------:|:-----------------:|:-----------------------------:|:-----------------------------:|:------------:
**Fe**  |3                          |0                  |-1                             |-1                             |0
**O**   |4                          |3                  |-6                             |-9                             |-1
**H**   |0                          |1                  |0                              |0                              |-2
**N**   |0                          |1                  |-2                             |-3                             |0


The `add_to_matrix` function updates the `element_matrix` with the **_count_** of a specific **_element_** for a given compound.

In [4]:
# Add the elements to the matrix
def add_to_matrix(element, index, side, count):
    if index == len(element_matrix):
        element_matrix.append([0] * len(element_list))

    if element not in element_list:
        element_list.append(element)
        for i in range(len(element_matrix)):
            element_matrix[i].append(0)

    column = element_list.index(element)
    element_matrix[index][column] += count * side

The `find_elements` function decomposes a **_chemical group_** into its constituent **_elements_** and their **_counts_**, then adds them to the matrix using the `add_to_matrix` function.

In [5]:
# Find the elements in a decomposed group
def find_elements(group, index, side, multiplier):
    elements = re.split("([A-Z][a-z]?)", group)
    # Filter out empty strings
    # re.split produces empty strings at the beginning and end of the list
    elements = [element for element in elements if element]
    elements.append('')

    i = 0
    while i < len(elements) and elements[i]:
        count = multiplier
        if elements[i + 1].isdigit():
            count *= int(elements[i + 1])
        
        add_to_matrix(elements[i], index, side, count)

        i += 2 if elements[i + 1].isdigit() else 1

The `decompose_compound` function breaks down a **_compound_** into **_smaller groups_** and processes each group to extract elements and their counts using the `find_elements` function.

In [6]:
# Decompose the compound into elements and their respective count 
def decompose_compound(compound, index, side):
    groups = re.split(r"(\([A-Za-z0-9]*\)[0-9]*)", compound)
    groups = [group for group in groups if group]

    for group in groups:
        multiplier = 1
        if group.startswith("("):
            group = re.split(r"\)([0-9]*)", group)
            if group[1].isdigit():
                multiplier = int(group[1])
            group = group[0][1:]

        find_elements(group, index, side, multiplier)

Add the **_elements_** to the **_matrix_** using the `decompose_compound` function.

In [7]:
for i in range(len(reactants)):
    decompose_compound(reactants[i], i, 1)
for i in range(len(products)):
    decompose_compound(products[i], i + len(reactants), -1)

print(f"Element list: {element_list}")
print(f"Element matrix: {element_matrix}")

Element list: ['Fe', 'O', 'H', 'N']
Element matrix: [[3, 4, 0, 0], [0, 3, 1, 1], [-1, -6, 0, -2], [-1, -9, 0, -3], [0, -1, -2, 0]]


### Chemical Equation Balanced Using Gaussian Elimination

The table above forms a **_system of linear equations_**, expressed as a **_matrix equation_** which needs solving: 

$$
Ax = 0
$$

where **A** is the coefficient matrix, and **x** is the vector of unknown stoichiometric coefficients.

#### Step 1: Convert to Augmented Matrix Form

Since the equation is homogeneous ($Ax = 0$), a **_zero column vector_** is appended to the matrix:

$$
\begin{bmatrix}
3 & 0 & -1 & -1 & 0 \\
4 & 3 & -6 & -9 & -1 \\
0 & 1 & 0 & 0 & -2 \\
0 & 1 & -2 & -3 & 0
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5
\end{bmatrix}
=
\begin{bmatrix}
0 \\
0 \\
0 \\
0
\end{bmatrix}
$$

#### Step 2: Transform to Reduced Row Echelon Form (RRRF)

**_Gaussian elimination_** is applied to transform **A** into an **_RREF matrix_**, where:

-   All rows having **_only zero entries_** are at the **_bottom_**.
-   The **_leading entry_** (the left-most nonzero entry) of every nonzero row, called the **_pivot_**, is on the **_right_** of the leading entry of every row above.
-   The **_leading entry_** in each nonzero row is **_1_** (called a **_leading one_**).
-   Each **_column_** containing a **_leading one_** has **_zeros_** in all its other entries.

After **_row reduction_**, the matrix should have a form of:

$$
\begin{bmatrix}
1 & 0 & 0 & 0 & b_1 \\
0 & 1 & 0 & 0 & b_2 \\
0 & 0 & 1 & 0 & b_3 \\
0 & 0 & 0 & 1 & b_4
\end{bmatrix}
$$

In this case, the coefficient matrix should be:

$$
\begin{bmatrix}
1 & 0 & 0 & 0 & -\frac{1}{4} \\
0 & 1 & 0 & 0 & -2 \\
0 & 0 & 1 & 0 & -\frac{1}{4} \\
0 & 0 & 0 & 1 & -\frac{1}{2}
\end{bmatrix}
$$

In [8]:
# Balance the equation
sympy_element_matrix = Matrix(element_matrix).transpose()

print(f"Element matrix: {element_matrix}")

Element matrix: [[3, 4, 0, 0], [0, 3, 1, 1], [-1, -6, 0, -2], [-1, -9, 0, -3], [0, -1, -2, 0]]


### Step 3: Extract the Solution

Since the system is **_underdetermined_** (more variables than equations), some variables have to be expressed **_in terms of others_**:

$$
x_1 = \frac{1}{4} x_5, \quad x_2 = 2 x_5, \quad x_3 = \frac{1}{4} x_5, \quad x_4 = \frac{1}{2} x_5
$$

A **_free variable_** is then chosen (_e.g._ $x_5 = 1$) and substitute back to solve for integer values for all coefficients.

In [9]:
# Get the nullspace of the matrix (Solve the system of equations)
solution = sympy_element_matrix.nullspace()[0]

print(f"Solution: {solution}")

Solution: Matrix([[1/4], [2], [1/4], [1/2], [1]])


### Step 4: Integer Scaling

Since coefficients cannot be in **_fractions_** or **_decimals_**, they need scaling by multiplying them by the **_least common multiple_** (**_LCM_**) of denominators to obtain the **_smallest integer solution_**.

In this case, the solution from step 3 is:

$$
\left(\frac{1}{4}, 2, \frac{1}{4}, \frac{1}{2}, 1\right)
$$

we multiply by 4 to get:

$$
\left(1,8,1,2,4\right)
$$

In [10]:
multiple = lcm([val.q for val in solution])
# Make all the coefficients integers
solution *= multiple

print(f"Solution: {solution}")

Solution: Matrix([[1], [8], [1], [2], [4]])


which are the final stoichiometric coefficients, resulting in the balanced equation:

$$
Fe_3O_4 + 8HNO_3 \rightarrow Fe(NO_3)_2 + 2Fe(NO_3)_3 + 4H_2O
$$

In [11]:
# Print the balanced equation
coefficients = solution.tolist()
result = ""

for i in range(len(reactants)):
    result += str(coefficients[i][0]) + reactants[i]
    if i < len(reactants) - 1:
        result += " + "

result += " -> "

for i in range(len(products)):
    result += str(coefficients[i + len(reactants)][0]) + products[i]
    if i < len(products) - 1:
        result += " + "

print(result)

1Fe3O4 + 8HNO3 -> 1Fe(NO3)2 + 2Fe(NO3)3 + 4H2O


## Demonstration

The following examples illustrate chemical equations that have been balanced using the algorithm:

In [12]:
def balance_equation(equation):
    global element_list, element_matrix
    
    element_list = []
    element_matrix = []

    # Get the chemical equation
    equation = re.sub(r"[ \[\]]", "", equation).split("->")

    # Get the reactants and products from the equation
    reactants = equation[0].split("+")
    products = equation[1].split("+")

    for i in range(len(reactants)):
        decompose_compound(reactants[i], i, 1)
    for i in range(len(products)):
        decompose_compound(products[i], i + len(reactants), -1)

    # Balance the equation
    sympy_element_matrix = Matrix(element_matrix).transpose()
    # Get the nullspace of the matrix (Solve the system of equations)
    solution = sympy_element_matrix.nullspace()[0]
    # Make all the coefficients integers
    multiple = lcm([val.q for val in solution])
    solution *= multiple

    # Print the balanced equation
    coefficients = solution.tolist()
    result = ""

    for i in range(len(reactants)):
        result += str(coefficients[i][0]) + reactants[i]
        if i < len(reactants) - 1:
            result += " + "

    result += " -> "

    for i in range(len(products)):
        result += str(coefficients[i + len(reactants)][0]) + products[i]
        if i < len(products) - 1:
            result += " + "

    print(result)

1.  **Equation:** $C_{57}H_{110}O_6 + O_2 \rightarrow CO_2 + H_2O$
    
    **Solution:** $2C_{57}H_{110}O_6 + 163O_2 \rightarrow 114CO_2 + 110H_2O$

In [13]:
balance_equation("C57H110O6+O2->CO2+H2O")

2C57H110O6 + 163O2 -> 114CO2 + 110H2O


2.  **Equation:** $C_{12}H_{22}O_{11} + KNO_3 \rightarrow K_2CO3 + N_2 + CO_2 + H_2O$

    **Solution:** $5C_{12}H_{22}O_{11} + 48KNO_3 \rightarrow 24K_2CO3 + 24N_2 + 36CO_2 + 55H_2O$

In [14]:
balance_equation("C12H22O11+KNO3->K2CO3+N2+CO2+H2O")

5C12H22O11 + 48KNO3 -> 24K2CO3 + 24N2 + 36CO2 + 55H2O


3.  **Equation:** $K_4[Fe(SCN)_6] + K_2Cr_2O_7 + H_2SO_4 \rightarrow Fe_2(SO_4)_3 + Cr_2(SO4)_3 + CO_2 + H_2O + K_2SO_4 + KNO_3$
    
    **Solution:** $6K_4[Fe(SCN)_6] + 97K_2Cr_2O_7 + 355H_2SO_4 \rightarrow 3Fe_2(SO_4)_3 + 97Cr_2(SO4)_3 + 36CO_2 + 355H_2O + 91K_2SO_4 + 36KNO_3$

In [15]:
balance_equation("K4[Fe(SCN)6]+K2Cr2O7+H2SO4->Fe2(SO4)3+Cr2(SO4)3+CO2+H2O+K2SO4+KNO3")

6K4Fe(SCN)6 + 97K2Cr2O7 + 355H2SO4 -> 3Fe2(SO4)3 + 97Cr2(SO4)3 + 36CO2 + 355H2O + 91K2SO4 + 36KNO3


## Collaboration

This project is a collaboration between:

-   **Ton Nu Thanh Thao**: [GitHub](https://github.com/thaoton1910) | [LinkedIn](https://www.linkedin.com/in/ton-nu-thanh-thao/) | [Gmail](mailto:thaoton1910@gmail.com)
-   **Khang Vu**: [GitHub](https://github.com/khangvum) | [LinkedIn](https://www.linkedin.com/in/khangvum/) | [Gmail](mailto:manhkhang0305@gmail.com)